**Using this template**

1. Change notebook title to assignment title you working on, eg. `Graded Challenge 1 or Milestone 1`
2. Put your identity such as name and batch number below notebook title.
3. Write your description before do any work. State your purpose of this assignment and what you are trying to achieve. 
4. Look at the rubric of this assignment and make sure you understand the criteria.
5. Code your solution in the cell provided below the working area section.
6. State your conlusions, findings, and any other relevant information in the cell provided below the conclusions sections.
7. Save this notebook and rename it to assignment title you working on  eg. `Graded Challenge 1 or Milestone 1`
8. Push your assignment before deadline.
9. Good Luck!

# Graded Challenge 2

1. Name: Jason Rich Darmawan Onggo Putra
2. Batch: 016 RMT

## Description

### Objective

`Write your objective here`

### Problem Statement

`Write your problem statement here`

## Working Area

```txt
Put your code, analysis, everything below this line

Make sure to check everything the rubric requires before you submit
```

## Data Retrieve

1. project: `bigquery-public-data`
2. dataset: [austin_waste](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m4!1m3!3m2!1sbigquery-public-data!2saustin_waste)
3. table: [waste_and_diversion](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m5!1m4!4m3!1sbigquery-public-data!2saustin_waste!3swaste_and_diversion)

**The query**:
```
SELECT report_date, load_time, load_type, load_weight, dropoff_site 
FROM `bigquery-public-data.austin_waste.waste_and_diversion`
WHERE report_date <= '2021-07-09';
```

## Import Library

In [1]:
import pandas as pd

## Data Loading

In [2]:
df = pd.read_csv("./h8dsft_P0GC2_jason_rich_darmawan_onggo_putra.csv")

In [3]:
df.head()

,report_date,load_time,load_type,load_weight,dropoff_site
0,2009-08-07,2009-08-07 11:51:00 UTC,RECYCLING - SINGLE STREAM,4400.0,MRF
1,2008-04-11,2008-04-11 12:31:00 UTC,RECYCLING - PAPER,1920.0,MRF
2,2009-01-16,2009-01-16 12:58:00 UTC,RECYCLING - SINGLE STREAM,4140.0,MRF
3,2009-07-10,2009-07-10 11:21:00 UTC,RECYCLING - SINGLE STREAM,4020.0,MRF
4,2006-02-03,2006-02-03 12:36:00 UTC,RECYCLING - COMINGLE,1700.0,MRF


In [4]:
df.tail()

,report_date,load_time,load_type,load_weight,dropoff_site
740868,2016-05-26,2016-05-26 12:00:00 UTC,RECYCLING - SINGLE STREAM,16260.0,AUSTIN WOOD RECYCLING
740869,2021-02-03,2021-02-03 02:05:00 UTC,RECYCLING - SINGLE STREAM,2340.0,AUSTIN WOOD RECYCLING
740870,2019-05-08,2019-05-08 11:18:00 UTC,RECYCLING - SINGLE STREAM,9110.0,AUSTIN WOOD RECYCLING
740871,2017-11-08,2017-11-08 01:15:00 UTC,RECYCLING - SINGLE STREAM,15960.0,AUSTIN WOOD RECYCLING
740872,2018-11-21,2018-11-21 01:10:00 UTC,RECYCLING - SINGLE STREAM,7100.0,AUSTIN WOOD RECYCLING


## Data Cleaning

## Data Exploration

## Conclusions, Assumptions, Overall Analysis

`put your conclusions here`